In [1]:
import pandas as pd
import numpy as np
import scipy as sp

In [2]:
import seaborn as sns
from matplotlib import pyplot as plt

In [93]:
class UGMM(object):
    
    def __init__(self, X, K=1):
        self.X = X
        self.K = K
        self.N = self.X.shape[0]

    def _init(self):
#         self.alpha=np.random.rand(self.K,1)
        self.m_prior= np.zeros((self.K,1))
        self.m=self.m_prior.copy()
        self.beta_prior=np.random.rand(self.K,1)
        self.beta=self.beta_prior.copy()
        self.nu_prior=np.ones((self.K,1))*np.random.randint(1,10)
        self.nu= self.nu_prior.copy()
        self.W_prior=np.ones((self.K,1))*np.random.randint(1,10)
        self.W=self.W_prior.copy()
        self.alpha_prior=np.ones((self.K,1))*1
        self.alpha=self.alpha_prior.copy()
        self.r=np.zeros((self.N,self.K))
        self.x_nebla=np.zeros((self.K,1))
        self.Nk=np.zeros((self.K,1))
        self.S=np.zeros((self.K,1))
        self.pi_nebla=np.zeros((self.K,1))
        self.lamda_nebla=np.zeros((self.K,1))
        self.E_mu_lamda=np.zeros((self.N,self.K))
        
        

        print('Init mean')
        print(self.m)
        

    def E_Step(self):
        for n in range(self.N):
            for k in range(self.K):
                self.pi_nebla[k]=np.exp(sp.special.digamma(self.alpha[k])-sp.special.digamma(self.alpha.sum()))
                self.lamda_nebla[k]=np.exp(sp.special.digamma(self.nu[k]/2, out=None)+np.log(2)+np.log(self.W[k]))
                self.E_mu_lamda[n][k]=(self.beta[k]**(-1))+self.nu[k]*self.W[k]*(self.X[n]-self.m[k])**2
                self.r[n][k]=self.pi_nebla[k]*(self.lamda_nebla[k]**0.5)*(np.exp(-0.5*self.E_mu_lamda[n][k]))
        row_sums = self.r.sum(axis=1)
        self.r= self.r / row_sums[:, np.newaxis]
        
        
    def fit(self, max_iter=1000, tol=1e-8):
        self._init()
#         self.r_values = [self.r_elbo()]
        self.m_history = [self.m.copy()]
        self.E_Step()
#         self.s2_history = [self.s2]
        for iter_ in range(1, max_iter+1):
#             print('before')
#             print(self.m_history)
            self.M_Step()
#             print('after')
#             print(self.m_history)
            self.m_history.append(self.m.copy())
            if iter_ % 25 == 0:
                print(iter_, self.m_history[iter_])
                
            if np.linalg.norm(self.m_history[-1]-self.m_history[-2],2) <= tol:
                print('ELBO converged with ll %.3f at iteration %d'%(self.m_history[-1][0],iter_))
                break
            self.E_Step()
        if iter_ == max_iter:
            print('ELBO ended with ll %.3f'%(self.m_history[-1][0]))


    def M_Step(self):
        self._update_Nk()
        self._update_x_nebla()
        self._update_S()
        self._update_alpha()
        self._update_W()
        self._update_beta()
        self._update_nu()
        self._update_m()

    def _update_Nk(self):
        for k in range(self.K):
            self.Nk[k]=self.r[:,k].sum()
            
    def _update_x_nebla(self):
        for k in range(self.K):
            self.x_nebla[k]=0
            for n in range(self.N):
#                 print(self.Nk)
                self.x_nebla[k]+=self.r[n][k]*self.X[n]/self.Nk[k]
    
    def _update_S(self):
        for k in range(self.K):
            self.S[k]=0
            for n in range(self.N):
                self.S[k]+=(self.r[n][k]*(self.X[n]-self.x_nebla[k])**(2))/self.Nk[k]
                
    def _update_alpha(self):
        for k in range(self.K):
            self.alpha[k]=self.alpha_prior[k]+self.Nk[k]
            
    def _update_beta(self):
        for k in range(self.K):
            self.beta[k]=self.beta_prior[k]+self.Nk[k]
        
    def _update_m(self):
        for k in range(self.K):
            self.m[k]=(self.beta_prior[k]*self.m_prior[k]+self.Nk[k]*self.x_nebla[k])/self.beta[k]
#         print('achha')
#         print(self.m)
    
    
    def _update_W(self):
        for k in range(self.K):
            self.W[k]=((1/self.W_prior[k])+self.Nk[k]*self.S[k]+(self.beta_prior[k]*self.Nk[k]/(self.beta_prior[k]+self.Nk[k]))*(self.x_nebla[k]-self.m_prior[k])**2)**(-1)
        
    
    def _update_nu(self):
        for k in range(self.K):
            self.nu[k]=self.nu_prior[k]+self.Nk[k]
    
#     def _calculate_rnk
        

In [94]:
data=pd.read_csv('data2.txt',header=None)

In [95]:
X =np.array(data.iloc[0,0])
for i in range(1,1000):
    X= np.append(X, data.iloc[i,0])
    

In [96]:
ugmm = UGMM(X, 3)
ugmm.fit()

Init mean
[[0.]
 [0.]
 [0.]]
25 [[0.45152632]
 [0.44588347]
 [0.45290098]]
50 [[0.4520092 ]
 [0.43676633]
 [0.45469518]]
75 [[0.45333965]
 [0.41815265]
 [0.45732545]]
100 [[0.46201477]
 [0.33582012]
 [0.46691744]]
125 [[0.50575608]
 [0.28077208]
 [0.51316456]]
150 [[0.51650332]
 [0.28743609]
 [0.53494457]]
175 [[0.51166521]
 [0.29142441]
 [0.54999722]]
200 [[0.49490892]
 [0.29507406]
 [0.56117519]]
225 [[0.47699133]
 [0.29848297]
 [0.5654122 ]]
250 [[0.46596113]
 [0.30057405]
 [0.56482317]]
275 [[0.45862642]
 [0.30185644]
 [0.56283554]]
300 [[0.45191086]
 [0.30274134]
 [0.56074303]]
325 [[0.44377604]
 [0.30334183]
 [0.55888669]]
350 [[0.42975374]
 [0.30365524]
 [0.55736716]]
375 [[0.38199985]
 [0.30362504]
 [0.55614288]]
400 [[0.10234284]
 [0.30288918]
 [0.55458015]]
425 [[0.02910261]
 [0.30273172]
 [0.55444949]]
450 [[0.02910323]
 [0.30272688]
 [0.55444276]]
475 [[0.0291035 ]
 [0.30272483]
 [0.55443992]]
500 [[0.02910361]
 [0.30272396]
 [0.55443872]]
525 [[0.02910366]
 [0.30272359]
 [

In [97]:
mean=ugmm.m

In [98]:
mean

array([[0.02910367],
       [0.30272349],
       [0.55443807]])